# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: distilbert-base-uncased
* Evaluation approach: I am evaluating classification accuracy by comparing predicted and actual labels.
* Fine-tuning dataset: rkf2778/amazon_reviews_mobile_electronics

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

my_model_name = "distilbert-base-uncased"

id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

# Criteria #1. Load a pretrained HF model
model = AutoModelForSequenceClassification.from_pretrained(
    my_model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from datasets import load_dataset

# Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(my_model_name)


# Dataset to be used: Amazon reviews of mobile electronics
my_dataset_name = "rkf2778/amazon_reviews_mobile_electronics"
splits = ["test", "train"]

# Criteria #2a Load dataset
datasets = {
    split: dataset
    for split, dataset in zip(splits, load_dataset(my_dataset_name, split=splits))
}

# Reduce the number of data for efficiency
for split in splits:
    datasets[split] = datasets[split].shuffle(seed=50).select(range(500))
    

train_dataset = datasets["train"]
test_dataset = datasets["test"]
    
# Criteria 2b: Preprocess the dataset inlcuding the star_rating_label representing the sentiment
def preprocess_review(batch):
    tokenized = tokenizer(batch["review_body"], padding=True, truncation=True, max_length=512)
    tokenized["labels"] = [label2id[label] for label in batch["star_rating_label"]]
    return tokenized

# Tokenize the datasets
tokenized_train = train_dataset.map(preprocess_review, batched=True)
tokenized_test = test_dataset.map(preprocess_review, batched=True)


In [7]:
import numpy as np
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Accuracy calculation function and Trainer setup is from Udacity lesson 4.14
def calculate_accuracy(predictions):
    logits, labels = predictions
    predicted_class = np.argmax(logits, axis=-1)
    return {"accuracy": (predicted_class == labels).mean()}


# Reference:  Udacity lession 4.14
training_args = TrainingArguments(
    output_dir="./data/original_model_sentiments_output",
    learning_rate=2e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

my_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create a trainer (referenced from Udacity lession 4.14)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=my_data_collator,
    compute_metrics=calculate_accuracy
)

trainer.train()

# Criteria #3: Evaluate the pretrained model
trainer.evaluate()


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.669815,0.644000


Checkpoint destination directory ./data/original_model_sentiments_output/checkpoint-63 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.6698151230812073,
 'eval_accuracy': 0.644,
 'eval_runtime': 2.6141,
 'eval_samples_per_second': 191.268,
 'eval_steps_per_second': 24.1,
 'epoch': 1.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [11]:
from peft import get_peft_model, LoraConfig, TaskType


# Reference: https://huggingface.co/docs/peft/package_reference/lora
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"], # Layers specific to distilbert
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS
)

# Criteria #4: Create a PEFT model
peft_model = get_peft_model(model, peft_config)

for param in peft_model.parameters():
    param.requires_grad = True

peft_model.print_trainable_parameters()


trainable params: 67,694,596 || all params: 67,694,596 || trainable%: 100.0


In [9]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Criteria #5: Train the PEFT model
peft_training_args = TrainingArguments(
    output_dir="./data/peft_model_sentiments_output_2",
    learning_rate=2e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=my_data_collator,
    compute_metrics=calculate_accuracy
)

peft_trainer.train()

# Criteria #6: Save the PEFT model 
peft_model.save_pretrained("my_peft_lora_distilbert2")


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.663358,0.644000
2,No log,0.653831,0.644000
3,No log,0.651145,0.644000


Checkpoint destination directory ./data/peft_model_sentiments_output_2/checkpoint-63 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/peft_model_sentiments_output_2/checkpoint-126 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/peft_model_sentiments_output_2/checkpoint-189 already exists and is non-empty.Saving will proceed but saved results may be invalid.


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [13]:
from peft import AutoPeftModelForSequenceClassification


# Criteria #7: Load the saved PEFT model (https://huggingface.co/docs/peft/package_reference/auto_class)
saved_peft_model = AutoPeftModelForSequenceClassification.from_pretrained("my_peft_lora_distilbert2")

# Recreate the PEFT trainer from the saved model
tuned_trainer = Trainer(
    model=saved_peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=my_data_collator,
    compute_metrics=calculate_accuracy
)


# Criteria #8: Compare and evaluate the fine tuned model
print(f"Original Model accuracy: {trainer.evaluate()}")
print(f"Fine-tuned Model accuracy: {tuned_trainer.evaluate()}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original Model accuracy: {'eval_loss': 0.6979907155036926, 'eval_accuracy': 0.356, 'eval_runtime': 2.6654, 'eval_samples_per_second': 187.592, 'eval_steps_per_second': 23.637, 'epoch': 1.0}


Fine-tuned Model accuracy: {'eval_loss': 0.6633461117744446, 'eval_accuracy': 0.644, 'eval_runtime': 2.7389, 'eval_samples_per_second': 182.555, 'eval_steps_per_second': 23.002}
